In [8]:
import pandas as pd

In [9]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
data.shape

(1460, 81)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [12]:
import numpy as np

class MyLabelEncoder:
    def __init__(self):
        self.label_dict = {}

    def fit(self, data):
        """
        This method creates a mapping of unique categories to numeric labels,
        while skipping NaN values.
        """
        unique_labels = set(data) - {np.nan}  # Exclude NaN values
        self.label_dict = {label: idx for idx, label in enumerate(unique_labels)}
        print(f"Label encoding dictionary (excluding NaN): {self.label_dict}")

    def transform(self, data):
        """
        This method converts the categorical data into numeric labels,
        skipping NaN values (NaN remains as NaN).
        """
        return [self.label_dict.get(label, np.nan) for label in data]

    def fit_transform(self, data):
        """
        This method combines fit and transform into one method.
        """
        self.fit(data)
        return self.transform(data)

In [13]:
encoder= MyLabelEncoder()
for col in data.select_dtypes(include=['object']).columns:
    data[col] = encoder.fit_transform(data[col])

Label encoding dictionary (excluding NaN): {'C (all)': 0, 'RH': 1, 'RM': 2, 'RL': 3, 'FV': 4}
Label encoding dictionary (excluding NaN): {'Grvl': 0, 'Pave': 1}
Label encoding dictionary (excluding NaN): {'Grvl': 0, 'Pave': 1}
Label encoding dictionary (excluding NaN): {'IR1': 0, 'Reg': 1, 'IR2': 2, 'IR3': 3}
Label encoding dictionary (excluding NaN): {'Low': 0, 'Bnk': 1, 'HLS': 2, 'Lvl': 3}
Label encoding dictionary (excluding NaN): {'NoSeWa': 0, 'AllPub': 1}
Label encoding dictionary (excluding NaN): {'FR2': 0, 'FR3': 1, 'CulDSac': 2, 'Corner': 3, 'Inside': 4}
Label encoding dictionary (excluding NaN): {'Gtl': 0, 'Sev': 1, 'Mod': 2}
Label encoding dictionary (excluding NaN): {'NoRidge': 0, 'NAmes': 1, 'NWAmes': 2, 'Blmngtn': 3, 'SawyerW': 4, 'MeadowV': 5, 'NPkVill': 6, 'BrkSide': 7, 'SWISU': 8, 'IDOTRR': 9, 'Blueste': 10, 'OldTown': 11, 'Veenker': 12, 'Somerst': 13, 'Timber': 14, 'NridgHt': 15, 'StoneBr': 16, 'Sawyer': 17, 'CollgCr': 18, 'Mitchel': 19, 'Gilbert': 20, 'Crawfor': 21, 'E

In [14]:
X = data.drop(columns=['Id', 'SalePrice']).fillna(25).values
y = data['SalePrice'].values

In [20]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
    ])

model.compile(optimizer='adam', loss='mse', metrics=['mae']) # Mean Squared Error and Mean Absolute Error
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1) # Adjust epochs and batch_size

loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Mean Absolute Error on test data: {mae}")

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 36091580416.0000 - mae: 175314.3906 - val_loss: 33750439936.0000 - val_mae: 168768.1719
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 29344497664.0000 - mae: 153650.1406 - val_loss: 16347784192.0000 - val_mae: 111038.8984
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 11475666944.0000 - mae: 81596.4609 - val_loss: 5376006656.0000 - val_mae: 50145.6094
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6651811840.0000 - mae: 48401.5781 - val_loss: 4243987200.0000 - val_mae: 43986.5625
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5445286912.0000 - mae: 45318.1992 - val_loss: 3739664896.0000 - val_mae: 41706.1680
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4624632320.0000 - mae: 40844.1133 - val_loss: 3255814912.0000 - val_mae: 39581.5117
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4850176512.0000 - mae: 44789.6562 - val_loss: 3058544896.0000 - val_mae: 38490.68

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on test data: {mae}")

Mean Absolute Error on test data: 21127.101271655796


In [23]:
data = data.fillna(25)
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,65.0,8450,1,25.0,1,3,1,...,0,25.0,25.0,25.0,0,2,2008,2,2,208500
1,2,20,3,80.0,9600,1,25.0,1,3,1,...,0,25.0,25.0,25.0,0,5,2007,2,2,181500
2,3,60,3,68.0,11250,1,25.0,0,3,1,...,0,25.0,25.0,25.0,0,9,2008,2,2,223500
3,4,70,3,60.0,9550,1,25.0,0,3,1,...,0,25.0,25.0,25.0,0,2,2006,2,3,140000
4,5,60,3,84.0,14260,1,25.0,0,3,1,...,0,25.0,25.0,25.0,0,12,2008,2,2,250000


In [24]:
for dt in data.columns:
  print(f'maximum: {data[dt].max()}')
  print(f'minimum: {data[dt].min()}')
  print(f'mean: {data[dt].mean()}')

maximum: 1460
minimum: 1
mean: 730.5
maximum: 190
minimum: 20
mean: 56.897260273972606
maximum: 4
minimum: 0
mean: 2.8527397260273974
maximum: 313.0
minimum: 21.0
mean: 62.05821917808219
maximum: 215245
minimum: 1300
mean: 10516.828082191782
maximum: 1
minimum: 0
mean: 0.9958904109589041
maximum: 25.0
minimum: 0.0
mean: 23.46986301369863
maximum: 3
minimum: 0
mean: 0.7102739726027397
maximum: 3
minimum: 0
mean: 2.8054794520547945
maximum: 1
minimum: 0
mean: 0.9993150684931507
maximum: 4
minimum: 0
mean: 3.554109589041096
maximum: 2
minimum: 0
mean: 0.09794520547945205
maximum: 24
minimum: 0
mean: 11.715068493150685
maximum: 8
minimum: 0
mean: 6.598630136986301
maximum: 7
minimum: 0
mean: 5.9746575342465755
maximum: 4
minimum: 0
mean: 2.0445205479452055
maximum: 7
minimum: 0
mean: 5.058219178082192
maximum: 10
minimum: 1
mean: 6.0993150684931505
maximum: 9
minimum: 1
mean: 5.575342465753424
maximum: 2010
minimum: 1872
mean: 1971.267808219178
maximum: 2010
minimum: 1950
mean: 1984.865753